<a href="https://colab.research.google.com/github/dinoelT/CAL/blob/master/testScore_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import altair as alt
import pandas as pd
import requests
import codecs
from requests.exceptions import HTTPError
import openpyxl

In [16]:
#add export?format=xlsx after id
url = "https://docs.google.com/spreadsheets/d/1pUDOO5rROKI20sP5p5_2gVCMvsOoBoFzYUNtz2fZYqA/export?format=xlsx"

In [17]:

try:
  #Send get request
  response = requests.get(url)
  # If the response was successful, no Exception will be raised
  response.raise_for_status()
except HTTPError as http_err:
  print(f'HTTP error occurred: {http_err}')
except Exception as err:
  print(f'Other error occurred: {err}')
else:
  print('Request successfully!')

Request successfully!


In [18]:
excelFileName = "excel.xlsx"

writeSuccessful = True
try:
  with codecs.open(excelFileName, 'wb') as f:
    f.write(response.content)
except Exception as err:
  print(f"Error occured when writing file: {err}")
  writeSuccessful = False
else:
  print("Excel file written successfully")


Excel file written successfully


In [19]:
#Get the sheet names in excel. 
sheetNames = openpyxl.load_workbook(excelFileName, read_only=True).sheetnames
print(f"Sheet names: {sheetNames}")

Sheet names: ['ИТБ', 'ФЭТ', 'ФМ']


In [20]:
if writeSuccessful:
  #Read pandas dataframes from Excel file that was saved
  dfs = [ pd.read_excel(excelFileName, header=1, sheet_name=sh) for sh in sheetNames]
else:
  #Read pandas dataframes from binary data stored in response.content
  dfs = [ pd.read_excel(response.content, header=1, sheet_name=sh) for sh in sheetNames]


In [21]:
#altair does not work with russian characters. Define sheet names in english
engSheetNames = ["ITMB", "FET", "FM"]

for i, sheetName in enumerate(engSheetNames):
  dfs[i]['Program'] = sheetName

In [22]:
#Concatenate the dataframes from all sheets
df = pd.concat(dfs, ignore_index=True)

#Drop the columns which have all nan values
df = df.dropna(axis='columns', how = 'all')

In [23]:
print(f"Russian column names: {list(df.columns)}")

Russian column names: ['№', 'ФИО', 'сумма баллов', 'Мат.', 'Русс.', 'ИКТ', 'Бюджет/договор', 'Program', 'Ин.яз']


In [24]:
filterColumns = ['ФИО', 'сумма баллов', 'Мат.', 'Русс.', 'ИКТ', 'Program', 'Ин.яз']

#Filter only the needed columns
df = df[filterColumns]

In [25]:
print(f"Russian column names: {list(df.columns)}")

Russian column names: ['ФИО', 'сумма баллов', 'Мат.', 'Русс.', 'ИКТ', 'Program', 'Ин.яз']


In [26]:
#Translate columns to english
englishColumns = ['Name', 'Total Score', 'Math', 'Russian', 'IT', 'Program', 'Foreign Language']

df.columns = englishColumns

In [27]:
df.head()

,Name,Total Score,Math,Russian,IT,Program,Foreign Language
0,Абрамов Петр Александрович,243,75.0,98.0,70.0,ITMB,NaN
1,Амерханова Наргиза Артуровна,286,96.0,89.0,96.0,ITMB,NaN
2,Баранов Алексей Алексеевич,235,45.0,96.0,94.0,ITMB,NaN
3,Баубеков Мирас Булатулы,217,70.0,81.0,66.0,ITMB,NaN
4,Богун Владислав Олегович,242,78.0,85.0,79.0,ITMB,NaN


In [28]:

alt.Chart(df).mark_circle(size=60).encode(
    x='Math',
    y='Russian',
    color='Program',
    tooltip=['Name', 'Total Score']
).interactive()

alt.Chart(...)